In [ ]:
import boto3
import json
import os

# Open AWS credentials saved in configuration.txt
with open('configuration.txt', 'r') as json_file:
    credentials = json.load(json_file)

region = credentials['region_name']
aws_access_key = credentials['aws_access_key_id']
aws_secret_key = credentials['aws_secret_access_key']

bucket_name = '<enter Bucket name>' # enter your S3 Bucket that you want to create on AWS

In [ ]:
# create S3 Client
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key,
                         aws_secret_access_key=aws_secret_key,
                         region_name=region)

In [ ]:
# create s3 Bucket 
response = s3_client.create_bucket(ACL='private', 
                                Bucket=bucket_name,
                                CreateBucketConfiguration={'LocationConstraint': 'ap-south-1'})
print(response)

In [ ]:
# Get the list of files that has to upload
files = os.listdir()

# upload file to s3 bucket 
for filename in files:
    response = s3_client.upload_file(filename, bucket_name, filename)
    print(response)
print('All files has been successfully uploaded.')

In [ ]:
# get the list of files that has been uploaded to S3 bucket 
response = s3_client.list_objects_v2(
    Bucket=bucket_name,
)

# delete the s3 bucket files one-by-one 
while response['KeyCount'] > 0:
    print('Deleting %d objects from bucket %s' % (len(response['Contents']),bucket_name))
    response = s3_client.delete_objects(
        Bucket=bucket_name,
        Delete={
            'Objects':[{'Key':obj['Key']} for obj in response['Contents']]
        }
    )
    response = s3_client.list_objects_v2(
        Bucket=bucket_name,
    )
# Delete the Bucket once it is empty
print('Now deleting bucket %s' % bucket_name)
response = s3_client.delete_bucket(
    Bucket=bucket_name
)